### <center>Zadanie 5</center>

#### Klasyfikacja na zbiorze dotyczącym stopnii otyłości

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, LearningCurveDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer
from sklearn.tree import DecisionTreeClassifier

### <center>Wczytanie i zapoznanie się z danymi</center>

In [ ]:
obesity = pd.read_csv('ObesityDataSet.csv')

obesity.drop(columns=['Weight'], inplace=True)

obesity.info()

In [ ]:
obesity.head()

#### <center>Opis kolumn</center>

<table>
    <tr>
        <th>Nazwa kolumny</th>
        <th>Opis</th>
        <th>Wartości</th>
    </tr>
    <tr>
        <td>Gender</td>
        <td>Płeć</td>
        <td>(Female/Male)</td>
    </tr>
    <tr>
        <td>Age</td>
        <td>Wiek pacjenta</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>Height</td>
        <td>Wzrost pacjenta</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>Weight</td>
        <td>Waga pacjenta</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>family_history_with_overweight</td>
        <td>Historia problemów z wagą w rodzinie</td>
        <td>yes/no</td>
    </tr>
    <tr>
        <td>FAVC</td>
        <td>Częste spożywanie wysokokalorycznych posiłków</td>
        <td>yes/no</td>
    </tr>
    <tr>
        <td>FCVC</td>
        <td>Częstotliwość spożycia warzyw</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>NCP</td>
        <td>Ilość głównych posiłków</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>CAEC</td>
        <td>Spożycie przekąsek między posiłkami</td>
        <td>Sometimes/Frequently/Always/no</td>
    </tr>
    <tr>
        <td>SMOKE</td>
        <td>Palenie papierosów</td>
        <td>yes/no</td>
    </tr>
    <tr>
        <td>CH2O</td>
        <td>Spożycie wody</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>SCC</td>
        <td>Śledzenie ilości spożytych kalorii</td>
        <td>yes/no</td>
    </tr>
    <tr>
        <td>FAF</td>
        <td>Częstotliwość aktywności fizycznej</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>TUE</td>
        <td>Czas poświęcony na korzystanie z urządzeń elektronicznych</td>
        <td>Zmienna numeryczna</td>
    </tr>
    <tr>
        <td>CALC</td>
        <td>Spożycie alkoholu</td>
        <td>Sometimes/Frequently/Always/no</td>
    </tr>
    <tr>
        <td>MTRANS</td>
        <td>Wykorzystywany środek transportu</td>
        <td>Public_Transportation/Automobile/Walking/Motorbike/Bike</td>
    </tr>
    <tr>
        <td>NObeyesdad</td>
        <td>Kategoria wagowa</td>
        <td>7 klas</td>
    </tr>
</table>


#### <center>Wykresy kołowe dla zmiennych kategorycznych</center>

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 16))
plt.suptitle('Podział pacjentów według kategorii', fontsize=20)

colors = plt.cm.Set3.colors
title_mapping = {
    'Gender': 'Płeć',
    'family_history_with_overweight': 'Historia rodzinna z nadwagą',
    'FAVC': 'Spożycie wysokokalorycznych posiłków',
    'CAEC': 'Spożycie przekąsek między posiłkami',
    'SMOKE': 'Palenie papierosów',
    'SCC': 'Śledzenie ilości spożytych kalorii',
    'CALC': 'Spożycie alkoholu',
    'MTRANS': 'Środek transportu'
}
object_columns = obesity.select_dtypes(include=['object']).columns[:-1]

for i, column in enumerate(object_columns):
    x, y = divmod(i, 2)
    el = obesity[column].value_counts()
    explode = [0.05 for _ in range(len(el))]

    if column == 'Gender':
        label_mapping = {'Female': 'Kobieta', 'Male': 'Mężczyzna'}
    elif column in ['CALC', 'CAEC']:
        label_mapping = {
            'Sometimes': 'Czasami',
            'no': 'Nigdy',
            'Frequently': 'Często',
            'Always': 'Zawsze'
        }
    elif column == 'MTRANS':
        label_mapping = {
            'Public_Transportation': 'Transport publiczny',
            'Automobile': 'Samochód',
            'Bike': 'Rower',
            'Walking': 'Chodzenie',
            'Motorbike': 'Motocykl'
        }
    else:
        label_mapping = {'yes': 'Tak', 'no': 'Nie'}

    translated_labels = [label_mapping.get(label) for label in el.index]

    ax[x, y].pie(
        x=el,
        explode=explode,
        labels=translated_labels,
        colors=colors,
        autopct='%1.1f%%',
        textprops={'fontsize': 12, 'fontweight': 'bold'},
        radius=1,
        startangle=180,
        labeldistance=1.1,
        wedgeprops={'edgecolor': 'white', 'linewidth': 2},
        normalize=True,
    )
    ax[x, y].set_title(title_mapping.get(column), fontsize=15, fontweight='bold')

ax[3, 1].axis('off')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### Kodowanie kategorii

- Dane z dwiema kategoriami: LabelEncoder
- Dane z większą liczbą kategorii: kodowanie one-hot

In [ ]:
many_categories = [col for col in object_columns if obesity[col].nunique() >= 3 and col != 'NObeyesdad']
binary_categories = [col for col in object_columns if obesity[col].nunique() <= 2 and col != 'NObeyesdad']
obesity = pd.get_dummies(obesity, columns=many_categories, drop_first=False)

obesity_encoder = LabelEncoder().fit(obesity['NObeyesdad'])
obesity_status = obesity['NObeyesdad']
obesity_status = obesity_encoder.transform(obesity_status)
obesity['NObeyesdad'] = obesity_status

encoders = [LabelEncoder() for _ in range(len(binary_categories))]
for i, column in enumerate(binary_categories):
    obesity[column] = encoders[i].fit_transform(obesity[column])
    obesity[column] = obesity[column].astype('category')

obesity.info()

#### <center>Histogramy dla danych numerycznych</center>

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(16, 12))
plt.suptitle('Rozkład wartości w kolumnach z zmiennymi liczbowymi', fontsize=20)

numeric_columns = obesity.select_dtypes(include=np.float64).columns

title_mapping = {
    'Age': 'Wiek',
    'Height': 'Wzrost',
    'Weight': 'Waga',
    'FCVC': 'Częstotliwość spożycia warzyw',
    'NCP': 'Liczba głównych posiłków',
    'CH2O': 'Spożycie wody',
    'FAF': 'Częstotliwość aktywności fizycznej',
    'TUE': 'Czas korzystania z urządzeń elektronicznych'
}

for i, column in enumerate(numeric_columns):
    x, y = divmod(i, 3)
    ax[x, y].hist(
        obesity[column],
        bins=20,
        color='#4C72B0',
        edgecolor='black',
        alpha=0.85
    )

    ax[x, y].set_title(f'{title_mapping.get(column)} ({column})', fontsize=14, fontweight='bold')
    ax[x, y].set_xlabel('Wartość', fontsize=11)
    ax[x, y].set_ylabel('Liczba obserwacji', fontsize=11)
    ax[x, y].grid(True, linestyle='--', alpha=0.6)

ax[2, 1].axis('off')
ax[2, 2].axis('off')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### <center>Macierz korelacji</center>

In [ ]:
plt.figure(figsize=(14, 10))
sns.set_theme(style='whitegrid')

corr = obesity.select_dtypes(include=[np.float64, np.int32]).corr()

sns.heatmap(
    corr,
    cmap='RdYlGn',
    annot=True,
    fmt='.2f',
    linewidths=0.5,
    linecolor='white',
    annot_kws={'fontsize': 11, 'fontweight': 'bold'},
    square=True,
    cbar_kws={'shrink': 0.8, 'label': 'Współczynnik korelacji'}
)

plt.title('Macierz korelacji zmiennych numerycznych', fontsize=16, pad=15)
plt.xticks(fontsize=12, rotation=45, ha='right')
plt.yticks(fontsize=12, rotation=0)
plt.tight_layout()
plt.show()

### <center>Klasyfikacja przy wykorzystaniu KNN, drzewa decyzyjnego i lasu</center>

In [ ]:
X = obesity.drop(columns='NObeyesdad')
y = obesity_status

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

X.head()

#### <center>Szukanie optymalnych hiperparametrów</center>

Optuna to framework służący do poszukiwania optymalnych hiperparametrów. W odróżnieniu od GridSearchCV nie przeszukuje wszystkich możliwych kombinacji parametrów aby odnaleźć najbardziej optymalne. Wykorzystuje strategie optymalizacji opartej na dedykowanych algorytmach, tzw. samplerach.

In [ ]:
def define_knn(trial):
    params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 3, 10),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree']),
        'leaf_size': trial.suggest_int('leaf_size', 5, 50),
    }
    return KNeighborsClassifier(**params)

def objective_knn(trial):
    knn = define_knn(trial)
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(knn, X_train, y_train, cv=skf, n_jobs=-1, scoring='accuracy')
    return scores.mean()

study_knn = optuna.create_study(direction='maximize', study_name='ObesityKNN', sampler=optuna.samplers.TPESampler())
study_knn.optimize(objective_knn, n_trials=10)

In [ ]:
def define_decision_tree(trial):
    params = {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 20),
    }
    return DecisionTreeClassifier(**params)

def objective_decision_tree(trial):
    dc = define_decision_tree(trial)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(dc, X_train, y_train, cv=skf, n_jobs=-1, scoring='accuracy')
    return scores.mean()

study_dc = optuna.create_study(direction='maximize', study_name='ObesityDecisionTree', sampler=optuna.samplers.TPESampler())
study_dc.optimize(objective_decision_tree, n_trials=50)

In [ ]:
def define_random_forest(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 20),
        'max_features': trial.suggest_int('max_features', 2, 20),
        'n_jobs': -1
    }
    return RandomForestClassifier(**params)

def objective_random_forest(trial):
    rf = define_random_forest(trial)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(rf, X_train, y_train, cv=skf, n_jobs=-1, scoring='accuracy')
    return scores.mean()

study_rf = optuna.create_study(direction='maximize', study_name='ObesityRandomForest', sampler=optuna.samplers.TPESampler())
study_rf.optimize(objective_random_forest, n_trials=100)

### <center>Najlepsze modele</center>

#### <center>1. KNN</center>

KNN (K-Nearest Neighbors) to prosty algorytm, który w klasyfikacji przypisuje etykietę na podstawie K najbliższych sąsiadów w przestrzeni cech. Nie buduje modelu w trakcie uczenia – decyzja zapada dopiero podczas predykcji na podstawie odległości (np. euklidesowej) między punktami.

In [ ]:
scaler_only_numeric = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns)
    ],
    remainder='passthrough'
)

bknn = define_knn(study_knn.best_trial)
knn_pipeline = Pipeline([('scaler_only_numeric', scaler_only_numeric), ('knn', bknn)])
knn_pipeline.fit(X_train, y_train)

class_labels = np.unique(y)
bknn_pred = knn_pipeline.predict(X_test)

print(classification_report(y_true=y_test, y_pred=bknn_pred, labels=class_labels, target_names=obesity_encoder.inverse_transform(class_labels)))

#### <center>2. Drzewo decyzyjne</center>

Drzewo decyzyjne to algorytm uczenia maszynowego, który posiada hierarchiczną strukturę, w której:

- węzły decyzyjne reprezentują warunki podziału danych na podstawie wartości określonych cech (atrybutów)
- gałęzie odpowiadają wynikom tych warunków
- liście zawierają końcowe przewidywania modelu

In [ ]:
bdc = define_decision_tree(study_dc.best_trial)
bdc.fit(X_train, y_train)
bdc_pred = bdc.predict(X_test)

print(classification_report(y_true=y_test, y_pred=bdc_pred, labels=class_labels, target_names=obesity_encoder.inverse_transform(class_labels)))

#### <center>Istotność cech dla drzewa decyzyjnego</center>

Ważność cech oparta na chaosie w danych (impurity-based feature importances). Chaos w danych oznacza jak bardzo dane są pomieszane na danym etapie drzewa.

Im wyższa wartość, tym ważniejsza cecha. Ważność cechy obliczana jest jako (znormalizowana) suma redukcji kryterium podziału, którą wnosi dana cecha w całym modelu. Ta metoda znana jest również jako Gini importance.

Mechanizm obliczania ważności cech:
- Drzewo buduje się, wybierając te cechy, które najlepiej zmniejszają impurity, np. rozdzielają klasy na czyste grupy.
- Dla każdej cechy liczy się, ile razy została użyta do podziału i o ile zmniejszyła impurity.
- Te wartości są sumowane dla drzewa i normalizowane (czyli przekształcone tak, aby ich suma wynosiła 1).

In [ ]:
dc_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': bdc.feature_importances_
}).sort_values(by='importance', ascending=True)

plt.figure(figsize=(8, 6))
sns.barplot(data=dc_feature_importance, x='importance', y='feature')

plt.title('Istotność cech dla drzewa decyzyjnego')
plt.xlabel('Istotność')
plt.ylabel('Cecha')
plt.tight_layout()
plt.show()

#### <center>3. Las losowy</center>

Las losowy to algorytm, który jest zbudowany z wielu drzew decyzyjnych. Każde z nich jest trenowane na innej losowej próbce danych treningowych. Wynik w klasyfikacji dla lasu jest osiągany przez głosowanie większościowe - wybierana jest klasa najczęściej wskazywana przez poszczególne drzewa. Jest bardziej odporny na przeuczenie niż pojedyńcze drzewo decyzyjne.

In [ ]:
brf = define_random_forest(study_rf.best_trial)
brf.fit(X_train, y_train)
brf_pred = brf.predict(X_test)

print(classification_report(y_true=y_test, y_pred=brf_pred, labels=class_labels, target_names=obesity_encoder.inverse_transform(class_labels)))

#### <center>Istotność cech dla lasu losowego</center>

In [ ]:
rf_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': brf.feature_importances_
}).sort_values(by='importance', ascending=True)

plt.figure(figsize=(8, 6))
sns.barplot(data=rf_feature_importance, x='importance', y='feature')

plt.title('Istotność cech dla lasu losowego')
plt.xlabel('Istotność')
plt.ylabel('Cecha')
plt.tight_layout()

plt.show()

#### <center>Macierze pomyłek</center>

Macierz pomyłek (błędów, ang. confusion matrix), inaczej określana mianem tablicy kontyngencji (ang. contingency table). Prezentuje liczby przypadków należących do poszczególnych poprawnych klas decyzyjnych oraz tych, które są przewidywane.

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(10, 18))

fig.suptitle('Macierze pomyłek dla wybranych klasyfikatorów', fontsize=16, fontweight='bold')

pred = [bknn_pred, bdc_pred, brf_pred]
titles = ['KNN', 'Drzewo decyzyjne', 'Las losowy']

sns.set_style('white')

for i in range(3):
    conf_matrix = confusion_matrix(y_test, pred[i])
    sns.heatmap(
        conf_matrix.T,
        annot=True,
        fmt='d',
        cbar=False,
        xticklabels=obesity_encoder.classes_,
        yticklabels=obesity_encoder.classes_,
        ax=ax[i],
        cmap='rocket'
    )
    ax[i].set_title(f'{titles[i]}', fontsize=16, pad=12)
    ax[i].set_xlabel('Rzeczywiste etykiety' if i == 2 else '')
    ax[i].set_ylabel('Przewidziane etykiety')
    ax[i].grid(False)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

#### <center>Krzywe ROC (One-vs-Rest)</center>
Dla problemu klasyfikacji na zbiorze, w którym jest więcej niż 2 klasy, bez przekształcenia klas nie można bezpośrednio wykorzystać krzywych ROC. Istnieją dwa popularne podejścia do adaptacji ROC do problemu wieloklasowego:

- One-vs-Rest (OvR) -> generuje N krzywych ROC, gdzie każda klasa jest traktowana jako pozytywna, a wszystkie pozostałe jako negatywne. To najczęściej stosowana metoda i umożliwia analizę skuteczności klasyfikatora względem każdej klasy osobno.

- One-vs-One (OvO) -> generuje N(N - 1)/2 krzywych ROC, porównując każdą parę klas oddzielnie (np. klasa 1 vs klasa 2, klasa 1 vs klasa 3, itd.). Ten sposób jest bardziej złożony, ale daje dokładniejszy wgląd w rozróżnialność między konkretnymi parami klas.

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)

class_of_interest = obesity_encoder.inverse_transform(class_labels)
knn_y_score = knn_pipeline.predict_proba(X_test)

fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 14))
fig.suptitle('Krzywe ROC One-vs-Rest dla KNN', fontsize=16, fontweight='bold')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#17becf']
knn_auc_scores = [roc_auc_score(y_onehot_test[:, i], knn_y_score[:, i]) for i in range(len(class_labels))]
legend_elements = [Line2D([0], [0], color=colors[i], label=f'{class_of_interest[i]} (AUC = {knn_auc_scores[i]:.3f})') for i in range(len(class_of_interest))]

for class_id in range(7):
    i, j = divmod(class_id, 2)
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        knn_y_score[:, class_id],
        name=f'{class_of_interest[class_id]} vs Rest',
        plot_chance_level=True,
        ax=ax[i, j],
        color=colors[class_id]
    )
    ax[i, j].grid(True)
    ax[i, j].set_label(class_of_interest[class_id])
    ax[i, j].set_title(f'{class_of_interest[class_id]} vs Rest', fontsize=14)
    ax[i, j].set_xlabel('False Positive Rate')
    ax[i, j].set_ylabel('True Positive Rate')
    ax[i, j].get_legend().remove()

ax[3, 1].axis('off')

fig.legend(
    title='Klasa (One-vs-Rest)',
    handles=legend_elements,
    bbox_to_anchor=(0.95, 0.22),
    ncol=2,
    fontsize=12,
    title_fontsize=13
)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)

class_of_interest = obesity_encoder.inverse_transform(class_labels)
dc_y_score = bdc.predict_proba(X_test)

fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 14))
fig.suptitle('Krzywe ROC One-vs-Rest dla drzewa decyzyjnego', fontsize=16, fontweight='bold')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#17becf']
dc_auc_scores = [roc_auc_score(y_onehot_test[:, i], dc_y_score[:, i]) for i in range(len(class_labels))]
legend_elements = [Line2D([0], [0], color=colors[i], label=f'{class_of_interest[i]} (AUC = {dc_auc_scores[i]:.3f})') for i in range(len(class_of_interest))]

for class_id in range(7):
    i, j = divmod(class_id, 2)
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        dc_y_score[:, class_id],
        name=f'{class_of_interest[class_id]} vs Rest',
        plot_chance_level=True,
        ax=ax[i, j],
        color=colors[class_id]
    )
    ax[i, j].grid(True)
    ax[i, j].set_label(class_of_interest[class_id])
    ax[i, j].set_title(f'{class_of_interest[class_id]} vs Rest', fontsize=14)
    ax[i, j].set_xlabel('False Positive Rate')
    ax[i, j].set_ylabel('True Positive Rate')
    ax[i, j].get_legend().remove()

ax[3, 1].axis('off')

fig.legend(
    title='Klasa (One-vs-Rest)',
    handles=legend_elements,
    bbox_to_anchor=(0.95, 0.22),
    ncol=2,
    fontsize=12,
    title_fontsize=13
)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)

class_of_interest = obesity_encoder.inverse_transform(class_labels)
rf_y_score = brf.predict_proba(X_test)

fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(12, 14))
fig.suptitle('Krzywe ROC One-vs-Rest dla lasu losowego', fontsize=16, fontweight='bold')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#17becf']
rf_auc_scores = [roc_auc_score(y_onehot_test[:, i], rf_y_score[:, i]) for i in range(len(class_labels))]
legend_elements = [Line2D([0], [0], color=colors[i], label=f'{class_of_interest[i]} (AUC = {rf_auc_scores[i]:.3f})') for i in range(len(class_of_interest))]

for class_id in range(7):
    i, j = divmod(class_id, 2)
    RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        rf_y_score[:, class_id],
        name=f'{class_of_interest[class_id]} vs Rest',
        plot_chance_level=True,
        ax=ax[i, j],
        color=colors[class_id]
    )
    ax[i, j].grid(True)
    ax[i, j].set_label(class_of_interest[class_id])
    ax[i, j].set_title(f'{class_of_interest[class_id]} vs Rest', fontsize=14)
    ax[i, j].set_xlabel('False Positive Rate')
    ax[i, j].set_ylabel('True Positive Rate')
    ax[i, j].get_legend().remove()

ax[3, 1].axis('off')

fig.legend(
    title='Klasa (One-vs-Rest)',
    handles=legend_elements,
    bbox_to_anchor=(0.95, 0.22),
    ncol=2,
    fontsize=12,
    title_fontsize=13
)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

#### <center>Krzywe uczenia</center>

Wykorzystano StratifiedKFold do wielokrotnego podziału przekazanych zbiorów X, y na zbiory treningowe i walidacyjne, z zachowaniem proporcji klas. Dzięki temu możliwe jest śledzenie, jak zmienia się wydajność modelu (np. dokładność lub strata) w zależności od liczby próbek treningowych. Krzywe uczenia pozwalają ocenić, czy model jest przeuczony (overfitting) lub niedouczony (underfitting).

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(12, 20))

plt.suptitle('Krzywe uczenia dla wybranych klasyfikatorów', fontsize=16, fontweight='bold')
models = [
    define_knn(study_knn.best_trial),
    define_decision_tree(study_dc.best_trial),
    define_random_forest(study_rf.best_trial)
]
titles = ['KNN', 'Drzewo decyzyjne', 'Las losowy']
for i in range(3):
    LearningCurveDisplay.from_estimator(
        estimator=models[i],
        X=X,
        y=y,
        cv=5,
        scoring='accuracy',
        ax=ax[i],
        n_jobs=-1
    )
    ax[i].grid(True)
    ax[i].set_title(titles[i], fontsize=16, pad=10)
    ax[i].set_ylabel('Wartość dokładności' if i == 2 else '')
    ax[i].set_xlabel('Liczba próbek')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()